In [1]:
import pandas as pd
from os.path import join
import glob

import matplotlib.pyplot as plt
%matplotlib inline

### Load the log file


In [2]:
log_df = pd.read_table("../../gutenberg-frozen-2018-07-18/.log",
                       names=["PGcode", "tokenizer", "lines_before", "lines_after", "tokens", "types"])
log_df.drop_duplicates(inplace=True)

In [3]:
n_log = len(log_df)

### Count number of files in /data

In [4]:
gutenberg_dir = "../../gutenberg-frozen-2018-07-18/"

In [9]:
n_raw = len(glob.glob(join(gutenberg_dir, "data", "raw", f"PG*_raw.txt")))
n_text = len(glob.glob(join(gutenberg_dir, "data", "text", f"PG*_text.txt")))
n_tokens = len(glob.glob(join(gutenberg_dir, "data", "tokens", f"PG*_tokens.txt")))
n_counts = len(glob.glob(join(gutenberg_dir, "data", "counts", f"PG*_counts.txt")))

In [10]:
n_raw

56002

In [11]:
n_text

55905

In [12]:
n_tokens

55905

In [13]:
n_counts

55905